## Access data from Redshift using the boto3 Redshift data api of AWS

### Input the SQL query to be run <span style="color:red">*(Change the SQL query with yours)*</span>

In [ ]:
### # SQL query without the comments
# simple example
query = "select * from <schema.table> limit 10;"


### Input parameters for the Redshift data API  <span style="color:green">*(No changes neeeded in this section)*</span>

In [ ]:
# input parameters passed from the caller event
# cluster identifier for the Amazon Redshift cluster
redshift_cluster_id = "<clustername>"
# database name for the Amazon Redshift cluster
redshift_database_name = "<dbname>"
# database user in the Amazon Redshift cluster with access to execute relevant SQL queries
redshift_user = "<username>"
# IAM Role of Amazon Redshift cluster having access to S3
redshift_iam_role = "arn:aws:iam::<iam role>"

### Define run_sql function using Redshift Data API to get query results into pandas dataframe  <span style="color:green">*(No changes neeeded in this section)*</span>

In [ ]:
import boto3
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

session = boto3.session.Session()
region = session.region_name


def run_sql(sql_text):
    # initiate redshift-data redshift_data_api_client in boto3
    redshift_data_api_client = boto3.client('redshift-data')
    
    # Initiate the query
    res = redshift_data_api_client.execute_statement(
            Database=redshift_database_name, DbUser=redshift_user, Sql=sql_text, ClusterIdentifier=redshift_cluster_id)
    
    
    query_id = res["Id"]
    done = False
    while not done:
        time.sleep(1)
        status_description = redshift_data_api_client.describe_statement(Id=query_id)
        status = status_description["Status"]
        if status == "FAILED":
            raise Exception('SQL query failed:' + query_id + ": " + status_description["Error"])
        elif status == "FINISHED":
            if status_description['ResultRows']>0:
                results = redshift_data_api_client.get_statement_result(Id=query_id)
                column_labels = []
                for i in range(len(results["ColumnMetadata"])): column_labels.append(results["ColumnMetadata"][i]['label'])
                records = []
                for record in results.get('Records'):
                    records.append([list(rec.values())[0] for rec in record])
                df = pd.DataFrame(np.array(records), columns=column_labels)
                return df
            else:
                return query_id

## Run your queries and save the results in a pandas dataframe

In [ ]:
df = run_sql(query);
df.head(10)

## References
1. https://github.com/aws-samples/getting-started-with-amazon-redshift-data-api/blob/main/quick-start/python/RedShiftDataAPI.py
2. https://github.com/aws-samples/getting-started-with-amazon-redshift-data-api/blob/main/use-cases/sagemaker-notebook-redshift-ml/redshift_ml_with_data_api.ipynb
3. https://docs.aws.amazon.com/redshift/latest/mgmt/data-api.html#data-api-calling-considerations

#### Note:
##### Alternate way of accessing Redshift using psycopg2 library, gives a timeout error.  However using the redshift data api is a good approach, as it uses the boto3 library from AWS and data can be accessed by IAM role instead of explicitly specifying the password